In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import os
import time
import string
import random
import os.path
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
driver = webdriver.Firefox(executable_path=r'./geckodriver')
driver.get('https://www.wto.org/english/news_e/spra_e/spra_e.htm')

In [5]:
newsArc = driver.find_element(By.ID, 'newsArc')
rows = newsArc.find_elements(By.CLASS_NAME, 'row')

In [10]:
df = []

for row in rows:
    title = row.find_element(By.TAG_NAME, 'h3').text
    print(title)
    for li in row.find_elements(By.TAG_NAME, 'li'):
        for link in li.find_elements(By.TAG_NAME, 'a'):
            type = link.text
            url = link.get_attribute('href')
            df.append([title, type, url])
            
df = pd.DataFrame(df, columns=['title', 'type', 'url'])

WTO report shows members moving to facilitate imports even as trade restrictions remain high
Informal Trade Policy Review Body meeting — trade monitoring report
DG Azevêdo: “We’ve achieved a lot, but much remains to be done”
DG Azevêdo: “Decisions made at the WTO will matter” for economic recovery from COVID-19 crisis
WTO members meet the candidates for Director-General
General Council Chair Walker announces timelines for next stages of DG selection process
DG Azevêdo: Tackling challenges facing MSMEs is critically important
DG Azevêdo to Ottawa Group: Cooperation at the WTO would help the global economy recover from COVID-19
DG Azevêdo and rules negotiations chair highlight importance of fisheries subsidies deal on World Oceans Day
WTO establishes nomination window for DG selection
Chancellor Merkel, DG Azevêdo and other agency heads discuss COVID-19 pandemic
DG Azevêdo announces he will step down on 31 August
IMF and WTO heads call for lifting trade restrictions on medical supplies a

In [9]:
title

'Troika recommends Carvalho de Azevêdo to be the next WTO Director-General'

In [316]:
tr.find_element(By.TAG_NAME, 'h3').text

'WTO Director-General Lamy to seek a second term'

In [8]:
df

,title,type,url
0,WTO report shows members moving to facilitate ...,News item,https://www.wto.org/english/news_e/news20_e/tr...
1,Informal Trade Policy Review Body meeting — tr...,Speech,https://www.wto.org/english/news_e/spra_e/spra...
2,"DG Azevêdo: “We’ve achieved a lot, but much re...",Speech,https://www.wto.org/english/news_e/spra_e/spra...
3,DG Azevêdo: “Decisions made at the WTO will ma...,News item,https://www.wto.org/english/news_e/news20_e/tn...
4,WTO members meet the candidates for Director-G...,News item,https://www.wto.org/english/news_e/news20_e/dg...
...,...,...,...
737,General Council appoints Azevêdo as next Direc...,Statement by the Chairman,https://www.wto.org/audio/2013_05_14_gc_bashir...
738,General Council appoints Azevêdo as next Direc...,Statement by Director-General Lamy,https://www.wto.org/audio/2013_05_14_gc_lamy_s...
739,General Council appoints Azevêdo as next Direc...,Statement by Ambassador Azevêdo,https://www.wto.org/audio/2013_05_14_gc_azeved...
740,Troika recommends Carvalho de Azevêdo to be th...,Statement by the Chair of the General Council,https://www.wto.org/english/news_e/news13_e/gc...


In [ ]:
path = os.path.abspath(f'../WTO_data_article/')
# print csv files in path
for file in os.listdir(path):
    print(file)
    